<a href="https://colab.research.google.com/github/davidlealo/TalkTrack/blob/main/TalkTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TalkTrack

In [1]:
!pip install opencv-python-headless numpy openai-whisper sounddevice


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=8a28b9dd03bf6fbf43c6442429d755125590f96f822cd5e4d24fa1cde6743416
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [ ]:
import cv2
import numpy as np
import whisper
import threading
import queue
import time
from datetime import datetime

# Initialize Whisper model
model = whisper.load_model("base")

# Queue to synchronize video frames and audio chunks
frame_queue = queue.Queue()
audio_queue = queue.Queue()

# Function to process video in real-time
def process_video():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Cannot access the camera")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Display the video frame
        cv2.imshow('TalkTrack - Video Feed', frame)

        # Add frame to the queue
        frame_queue.put((datetime.now(), frame))

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to process audio in real-time
def process_audio():
    import sounddevice as sd

    def callback(indata, frames, time, status):
        if status:
            print(status)
        # Add audio chunk to the queue
        audio_queue.put(indata.copy())

    # Set up audio stream
    with sd.InputStream(samplerate=16000, channels=1, callback=callback):
        print("Recording audio...")
        while True:
            time.sleep(0.1)

# Function to transcribe audio
def transcribe_audio():
    while True:
        if not audio_queue.empty():
            audio_chunk = audio_queue.get()
            # Convert audio chunk to Whisper-compatible format
            audio_data = np.frombuffer(audio_chunk, np.float32).flatten()
            result = model.transcribe(audio_data)
            print("Transcription:", result["text"])

# Start threads for video and audio
video_thread = threading.Thread(target=process_video)
audio_thread = threading.Thread(target=process_audio)
transcription_thread = threading.Thread(target=transcribe_audio)

video_thread.start()
audio_thread.start()
transcription_thread.start()

# Wait for threads to finish
video_thread.join()
audio_thread.join()
transcription_thread.join()


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 45.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Error: Cannot access the camera


Exception in thread Thread-12 (process_audio):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-16af1d1c90fe>", line 43, in process_audio
  File "/usr/local/lib/python3.10/dist-packages/sounddevice.py", line 71, in <module>
    raise OSError('PortAudio library not found')
OSError: PortAudio library not found
